In [1]:
import os
import datetime
os.chdir('..')
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib
from backtesting import Backtest, Strategy
from src.models.predict_model import PredictModel
from src.features.preprocess_pipeline import InferencePreprocessPipeline

In [31]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 640)

#### Prep Data

In [2]:
prepared_train_data = pd.read_csv("../data/processed/series_train_preprocessed.csv")
prepared_test_data = pd.read_csv("../data/processed/series_test_preprocessed.csv")
prepared_validation_data = pd.read_csv("../data/processed/series_validate_preprocessed.csv")

In [3]:
X_train = prepared_train_data.drop('target', axis=1)
y_train = prepared_train_data['target']

X_test = prepared_test_data.drop('target', axis=1)
y_test = prepared_test_data['target']

X_val = prepared_validation_data.drop('target', axis=1)
y_val = prepared_validation_data['target']

top_20_features_final = joblib.load("../artefacts/top_20_features_final.pkl")

X_train_20_final = X_train[top_20_features_final]
X_test_20_final = X_test[top_20_features_final]
X_val_20_final = X_val[top_20_features_final]

# reshape the data
def reshape_data(X_train, X_test, X_val):
    timesteps = 1
    X_train = X_train.values.reshape(-1, timesteps, len(X_train.columns))
    X_test = X_test.values.reshape(-1, timesteps, len(X_test.columns))
    X_val = X_val.values.reshape(-1, timesteps, len(X_val.columns))
    return X_train, X_test, X_val

X_train_20_final_gru, X_test_20_final_gru, X_val_20_final_gru = reshape_data(
    X_train_20_final,
    X_test_20_final,
    X_val_20_final)

In [4]:
os.chdir('..')

In [4]:
X_test_20_final.loc[[0],:]

,high_t,tema_14_t,wcl_price_t,med_price_t-2,med_price_t,close_t,typ_price_t,avg_price_t,tema_9_t,open_t,typ_price_t-1,wcl_price_t-2,dema_10_t-1,close_t-1,upperband_5_t,tema_14_t-1,middleband_5_t,ma_5_t,med_price_t-1,tema_9_t-1
0,0.227233,0.228873,0.229047,0.229325,0.229235,0.229233,0.228985,0.229024,0.22871,0.229556,0.228962,0.229329,0.228957,0.229616,0.225662,0.229066,0.228298,0.228298,0.229008,0.228929


#### Random Shit

In [25]:
stacker = PredictModel()


In [7]:
stacker.stack_predict(pd.DataFrame(X_test_20_final.loc[[2],:]))

1/1 [==============================] - 0s 26ms/step


1.0562469265871743

In [13]:
X_test_exp = X_test.copy()
new_cols = [col[:-2] if col.endswith('_t') else col for col in X_test.columns]
X_test_exp.columns = new_cols

In [17]:
X_test_exp.columns
# forecast = X_test_exp['close'].iloc[-1]
# forecast
# perc_diff = (forecast - self.data['close'].iloc[-1]) / self.data['close'].iloc[-1] * 100

Index(['open_t-6', 'high_t-6', 'low_t-6', 'close_t-6', 'tick_volume_t-6',
       'spread_t-6', 'real_volume_t-6', 'impact_t-6', 'sentiment_score_t-6',
       'adx_14_t-6',
       ...
       'atr_7', 'atr_21', 'natr_14', 'natr_7', 'natr_21', 'trange',
       'avg_price', 'med_price', 'typ_price', 'wcl_price'],
      dtype='object', length=945)

In [7]:
test_data_raw = pd.read_csv("./data/processed/test_preprocessed.csv")

In [19]:
test_data_raw.columns

Index(['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread',
       'real_volume', 'impact', 'sentiment_score',
       ...
       'atr_21', 'natr_14', 'natr_7', 'natr_21', 'trange', 'avg_price',
       'med_price', 'typ_price', 'wcl_price', 'target'],
      dtype='object', length=137)

In [20]:
scaler = joblib.load("../artefacts/scaler.pkl")
dummy = np.zeros((1, 136))

def __invTransform(scaler, data: float, dummy: np.ndarray, col_idx: int=3) -> float:
    data = np.asarray([data])
    dummy[:, col_idx] = data
    dummy = pd.DataFrame(scaler.inverse_transform(dummy))
    return dummy.iloc[:, col_idx].values[0]

c:\Users\ab6ri\OneDrive\Documents\Personal_Projects\simmer_bot\.venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [21]:
vanilla_test_data = pd.read_csv("../data/interim/parsed_scraped_data_clipped.csv")

In [62]:
len(vanilla_test_data['time']) - len(test_data_raw['close'])

243042

In [31]:
assert len(test_data_raw['close']) == len(vanilla_test_data['close'].iloc[-len(test_data_raw['close']):])

In [63]:
test_data_raw['time'].iloc[0]

'2023-10-16 23:30:00'

In [77]:
vanilla_test_data['close'].iloc[-(len(test_data_raw['close'])+199)]

1.05272

In [90]:
len(vanilla_test_data.iloc[-(len(test_data_raw['close'])):-6]) == len(X_test_20_final['close_t'])

True

In [68]:
last_close = __invTransform(scaler, test_data_raw['close'].iloc[-2], dummy, 3)
assert last_close == vanilla_test_data['close'].iloc[-7]
# last_close

In [81]:
ind =5000
precc_difff = (last_close - vanilla_test_data['close'].iloc[-ind]) / vanilla_test_data['close'].iloc[-ind] * 100
precc_difff = last_close - 0.02 * last_close
print("percentage difference: ", precc_difff, "%")

percentage difference:  1.0311952 %


In [75]:
1.119095 - 1.05575 

0.06334499999999998

#### Backtesting

In [20]:
raw_data = pd.read_csv("./data/interim/merged_scrapped_n_price.csv")
top_20_features_final = joblib.load("./artefacts/top_20_features_final.pkl")


In [4]:
# Len test data => 26977

In [17]:
data_points = 26977

In [21]:
backtest_raw_data = raw_data.iloc[-(data_points+205):,:]
backtest_raw_data.reset_index(drop=True, inplace=True)

In [6]:
model_data = InferencePreprocessPipeline(backtest_raw_data).run()[top_20_features_final]

Preparing time series data: 100%|█████████▉| 26977/26983 [00:14<00:00, 1919.00 rows/s]


In [7]:
model_data.head()

,high_t,tema_14_t,wcl_price_t,med_price_t-2,med_price_t,close_t,typ_price_t,avg_price_t,tema_9_t,open_t,typ_price_t-1,wcl_price_t-2,dema_10_t-1,close_t-1,upperband_5_t,tema_14_t-1,middleband_5_t,ma_5_t,med_price_t-1,tema_9_t-1
0,0.226715,0.229066,0.229126,0.229042,0.229008,0.229616,0.228962,0.229047,0.228929,0.229714,0.229203,0.229120,0.229005,0.229707,0.225750,0.229133,0.228438,0.228438,0.229325,0.228977
1,0.227233,0.228873,0.229047,0.229325,0.229235,0.229233,0.228985,0.229024,0.228710,0.229556,0.228962,0.229329,0.228957,0.229616,0.225662,0.229066,0.228298,0.228298,0.229008,0.228929
2,0.226265,0.228675,0.228719,0.229008,0.228714,0.229098,0.228593,0.228656,0.228506,0.229263,0.228985,0.229126,0.228788,0.229233,0.225717,0.228873,0.228202,0.228202,0.229235,0.228710
3,0.226355,0.228634,0.228917,0.229235,0.228794,0.229414,0.228751,0.228747,0.228537,0.229150,0.228593,0.229047,0.228614,0.229098,0.225669,0.228675,0.228171,0.228171,0.228714,0.228506
4,0.226355,0.228452,0.228657,0.228714,0.228703,0.228985,0.228548,0.228673,0.228355,0.229466,0.228751,0.228719,0.228587,0.229414,0.225519,0.228634,0.228025,0.228025,0.228794,0.228537


In [8]:
len(model_data)

26977

In [9]:
predictions = []
model = PredictModel()
for index, row in tqdm(model_data.iterrows(), total=len(model_data), unit=" rows"):
    predictions.append(model.stack_predict(pd.DataFrame(model_data.loc[[index],:])))

100%|██████████| 26977/26977 [29:31<00:00, 15.23 rows/s]


In [10]:
# save predictions
joblib.dump(
    predictions,
    "./data/processed/predictions.pkl"
)

['./data/processed/predictions.pkl']

In [22]:
backtest_raw_data.head()

,time,open,high,low,close,tick_volume,spread,real_volume,impact,link_text
0,2023-10-12 21:30:00,1.05342,1.05363,1.05299,1.05321,881,14,0,Low Impact Expected,Italian industrial production data has seen so...
1,2023-10-12 21:45:00,1.05320,1.05337,1.05250,1.05262,604,14,0,Low Impact Expected,Italian industrial production data has seen so...
2,2023-10-12 22:00:00,1.05262,1.05318,1.05258,1.05283,571,14,0,Low Impact Expected,Italian industrial production data has seen so...
3,2023-10-12 22:15:00,1.05282,1.05313,1.05280,1.05304,448,14,0,Low Impact Expected,Italian industrial production data has seen so...
4,2023-10-12 22:30:00,1.05302,1.05333,1.05293,1.05293,552,14,0,Low Impact Expected,Italian industrial production data has seen so...


In [23]:
proper_cols = [
    'time', 'Open', 'High', 'Low', 'Close', 'Volume', 'spread',
    'real_volume', 'impact', 'link_text']
bt_data = backtest_raw_data.copy()
bt_data = bt_data.iloc[-(data_points):,:]
bt_data.reset_index(drop=True, inplace=True)
bt_data.set_index(pd.to_datetime(bt_data['time']), drop=False, inplace=True)
bt_data.columns=proper_cols
bt_data.index.name=None

In [24]:
bt_data.Close.head(100)

2024-10-03 17:00:00    1.10257
2024-10-03 17:15:00    1.10276
2024-10-03 17:30:00    1.10129
2024-10-03 17:45:00    1.10163
2024-10-03 18:00:00    1.10177
2024-10-03 18:15:00    1.10117
2024-10-03 18:30:00    1.10180
2024-10-03 18:45:00    1.10137
2024-10-03 19:00:00    1.10201
2024-10-03 19:15:00    1.10241
2024-10-03 19:30:00    1.10293
2024-10-03 19:45:00    1.10295
2024-10-03 20:00:00    1.10342
2024-10-03 20:15:00    1.10310
2024-10-03 20:30:00    1.10334
2024-10-03 20:45:00    1.10321
2024-10-03 21:00:00    1.10296
2024-10-03 21:15:00    1.10322
2024-10-03 21:30:00    1.10282
2024-10-03 21:45:00    1.10252
2024-10-03 22:00:00    1.10255
2024-10-03 22:15:00    1.10302
2024-10-03 22:30:00    1.10282
2024-10-03 22:45:00    1.10276
2024-10-03 23:00:00    1.10257
2024-10-03 23:15:00    1.10278
2024-10-03 23:30:00    1.10276
2024-10-03 23:45:00    1.10292
2024-10-04 00:00:00    1.10301
2024-10-04 00:15:00    1.10301
2024-10-04 00:30:00    1.10302
2024-10-04 00:45:00    1.10306
2024-10-

In [25]:
def percentage_difference_in_pips(price1: float, price2: float, pip_size=0.0001) -> float:
    """
    Calculate the pole-sensitive percentage difference in pips between two prices.
    
    Parameters:
        price1 (float): The first price.
        price2 (float): The second price.
        pip_size (float): The size of a pip (default is 0.0001).
        
    Returns:
        float: The pole-sensitive percentage difference.
    """
    # Calculate the price difference
    difference = price2 - price1
    
    # Convert difference to pips
    pip_difference = difference / pip_size
    
    # Calculate percentage difference
    percentage_difference = (pip_difference / (price1 / pip_size)) * 100
    
    return percentage_difference


In [26]:
def pip_difference(price1, price2, pip_size=0.0001):
    """
    Calculate the pole-sensitive pip difference between two prices.
    
    Parameters:
        price1 (float): The first price.
        price2 (float): The second price.
        pip_size (float): The size of a pip (default is 0.0001).
        
    Returns:
        float: The pole-sensitive pip difference.
    """
    # Calculate the price difference
    difference = price1 - price2
    
    # Convert difference to pips
    pip_difference = difference / pip_size
    
    return pip_difference

In [58]:
class MyStrategy(Strategy):
    predictions = predictions
    sl_perc = 0.002
    tp_perc = 0.006
    buy_threshold = 0.004
    sell_threshold = 0.004
    close_days = 1

    def init(self):
        super().init()
        self.predictions = predictions
        self.idx = 0
        self.trade_size = 0.5



    def next(self):
        super().next()
        forecast = self.predictions[self.idx]

        current_price = self.data['Close'][-1]

        # Check and close trades open for more than 1 day
        for trade in self.trades:
            if self.data.index[-1] - trade.entry_time >= datetime.timedelta(days=int(self.close_days)):
                trade.close()
                # print("closed overdue trade >>", trade.value)
        
        # Buy logic
        if forecast > current_price * (1 + self.buy_threshold) and not self.position:
            sl = current_price * (1 - self.sl_perc)
            tp = forecast  # Use model's forecast as target price
            self.buy(size=self.trade_size, tp=tp, sl=sl)

        # Sell logic
        elif forecast < current_price * (1 - self.sell_threshold) and not self.position:
            sl = current_price * (1 + self.sl_perc)
            tp = forecast  # Use model's forecast as target price
            self.sell(size=self.trade_size, tp=tp, sl=sl)

        # increment the index
        self.idx += 1


In [59]:
bt = Backtest(data=bt_data, strategy=MyStrategy, cash=300, margin=1/5, commission=0.0002)
stats, heatmap = bt.optimize(
    close_days=[i for i in range(1, 3)],
    sl_perc=[i/1000 for i in range(1, 9)],  # Expanding the range for stop loss
    tp_perc=[i/1000 for i in range(1, 9)],  # Expanding the range for take profit
    buy_threshold=[i/1000 for i in range(1, 6)],  # Testing different buy thresholds
    sell_threshold=[i/1000 for i in range(1, 6)],  # Testing different sell thresholds
    maximize='Return [%]', max_tries=5000,
    random_state=0,
    return_heatmap=True
)

c:\Users\ab6ri\OneDrive\Documents\Personal_Projects\simmer_bot\.venv\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 3200 configurations.
  output = _optimize_grid()


  0%|          | 0/16 [00:00<?, ?it/s]

In [60]:
stats

Start                     2023-10-17 00:45:00
End                       2024-11-15 23:45:00
Duration                    395 days 23:00:00
Exposure Time [%]                   17.755866
Equity Final [$]                   315.205783
Equity Peak [$]                     318.46637
Return [%]                           5.068594
Buy & Hold Return [%]               -0.197958
Return (Ann.) [%]                    3.821141
Volatility (Ann.) [%]                7.187014
Sharpe Ratio                         0.531673
Sortino Ratio                        0.741469
Calmar Ratio                         0.743105
Max. Drawdown [%]                   -5.142125
Avg. Drawdown [%]                   -0.685669
Max. Drawdown Duration      226 days 05:30:00
Avg. Drawdown Duration       10 days 18:30:00
# Trades                                  107
Win Rate [%]                        68.224299
Best Trade [%]                       0.500103
Worst Trade [%]                     -0.623834
Avg. Trade [%]                    

In [61]:
heatmap.sort_values()

close_days  sl_perc  tp_perc  buy_threshold  sell_threshold
1           0.001    0.001    0.001          0.001            -45.271873
                     0.002    0.001          0.001            -45.271873
                     0.004    0.001          0.001            -45.271873
                     0.005    0.001          0.001            -45.271873
                     0.006    0.001          0.001            -45.271873
                                                                 ...    
            0.006    0.001    0.003          0.001              5.068594
                     0.002    0.003          0.001              5.068594
                     0.006    0.003          0.001              5.068594
                     0.008    0.003          0.001              5.068594
                     0.005    0.003          0.001              5.068594
Name: Return [%], Length: 3200, dtype: float64